In [31]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pose_dataset import PoseData

In [32]:
INPUT_SIZE = 33 * 3
HIDDEN_SIZE = 64
NUM_LAYERS = 2 # number of RNNs to stack
NUM_CLASSES = 9 # number of categories

LEARNING_RATE = 0.1

In [43]:
TIME_DIM = 1
BATCH_DIM = 0
COORD_DIM = 2

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
class PoseScoringModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(PoseScoringModel, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        h0 = torch.zeros(self.num_layers, X.size(BATCH_DIM), self.hidden_size).to(device)
        print(X.shape)
        out, _ = self.gru(X, h0)
        # out: batch x time x hidden
        out = out[:, -1, :]
        # out: batch x hidden
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

In [62]:
model = PoseScoringModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES)

In [63]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [64]:
NUM_WORKERS = 2

In [65]:
import os
import glob
import shutil
import random

# Set random seed for reproducibility (optional)
# random.seed(42)

# Set the training ratio (90% training, 10% testing)
train_ratio = 0.9

# Define target directories
train_dir = "../data/train"
test_dir = "../data/test"

# Create directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get a sorted list of all data files
data_files = sorted(glob.glob("../data/data_*.mat"))
data_files = data_files[:19]

for data_file in data_files:
    # Get the base filename without extension, e.g., "data_000"
    base_name = os.path.splitext(os.path.basename(data_file))[0]
    parts = base_name.split('_')
    if len(parts) != 2:
        print(f"Skipping invalid file name: {data_file}")
        continue
    number = parts[1]  # e.g., "000"
    
    # Construct the corresponding metric filename
    metric_file = f"../data/metric_{number}.mat"
    
    # Check if the corresponding metric file exists
    if not os.path.exists(metric_file):
        print(f"Warning: {metric_file} not found for {data_file}")
        continue

    # Choose destination based on random split
    destination = train_dir if random.random() < train_ratio else test_dir

    # Move both files
    shutil.move(data_file, os.path.join(destination, os.path.basename(data_file)))
    shutil.move(metric_file, os.path.join(destination, os.path.basename(metric_file)))
    
    print(f"Moved pair ({data_file}, {metric_file}) to {destination}")

Moved pair (../data/data_114.mat, ../data/metric_114.mat) to ../data/train
Moved pair (../data/data_115.mat, ../data/metric_115.mat) to ../data/train
Moved pair (../data/data_116.mat, ../data/metric_116.mat) to ../data/train
Moved pair (../data/data_117.mat, ../data/metric_117.mat) to ../data/test
Moved pair (../data/data_118.mat, ../data/metric_118.mat) to ../data/train
Moved pair (../data/data_119.mat, ../data/metric_119.mat) to ../data/test
Moved pair (../data/data_120.mat, ../data/metric_120.mat) to ../data/train
Moved pair (../data/data_121.mat, ../data/metric_121.mat) to ../data/train
Moved pair (../data/data_122.mat, ../data/metric_122.mat) to ../data/train
Moved pair (../data/data_123.mat, ../data/metric_123.mat) to ../data/train
Moved pair (../data/data_124.mat, ../data/metric_124.mat) to ../data/train
Moved pair (../data/data_125.mat, ../data/metric_125.mat) to ../data/train
Moved pair (../data/data_126.mat, ../data/metric_126.mat) to ../data/train
Moved pair (../data/data_12

In [66]:
train_data = PoseData("../data/train")
test_data = PoseData("../data/test")

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)
# train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
# test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

In [76]:
import os
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torchmetrics.classification import MulticlassF1Score

EPOCHS = 10  # Number of epochs for training
model.train()  # Set the model to training mode

# Create a directory to save snapshots if it doesn't exist
os.makedirs("snapshots", exist_ok=True)
# Number of splits for k-fold cross-validation
k_folds = 5
batch_size = 32
kf = KFold(n_splits=k_folds, shuffle=True)

indices = list(range(len(train_data)))

# Convert dataset to a tensor for splitting
for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
    print(f"Fold {fold + 1}/{k_folds}")

    train_subset = Subset(train_data, train_idx)
    val_subset = Subset(train_data, val_idx)
    
    # Create dataloaders for these subsets
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    
    
    for epoch in range(EPOCHS):
        model.train()
        for i, (inputs, targets) in enumerate(train_dataloader):
            if inputs.shape[0] != 32:
                continue
            inputs = inputs[:, :, :, :3].flatten(2)

            # Forward pass
            outputs = model(inputs)
            targets = targets.squeeze(1)
            loss_value = loss(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()

            if i % 10 == 0:
                print(f"Fold [{fold + 1}/{k_folds}], Epoch [{epoch + 1}/{EPOCHS}], "
                      f"Step [{i + 1}/{len(train_dataloader)}], Loss: {loss_value.item():.4f}")

        # Evaluate on validation set
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        f1_metric = MulticlassF1Score(num_classes=3, average='micro').to(device)
        with torch.no_grad():
            for inputs, targets in val_loader:
                if inputs.shape[0] != 32:
                    continue
                inputs, targets = inputs.to(device), targets.to(device)
                inputs = inputs[:, :, :, :3].flatten(2)

                logits = model(inputs)
                preds = torch.argmax(logits, dim=1)

                # Update the metric
                f1_metric.update(preds, targets)
                
        f1_score = f1_metric.compute()
        print(f"F1-score (micro): {f1_score.item():.4f}")
        f1_metric.reset()
        # Save a snapshot of the model at each epoch
        torch.save(model.state_dict(), f"snapshots/model_fold{fold + 1}_epoch{epoch + 1}.pth")

Fold 1/5
torch.Size([32, 36, 99])
Fold [1/5], Epoch [1/10], Step [1/4], Loss: 9.7518
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [2/10], Step [1/4], Loss: 9.4773
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [3/10], Step [1/4], Loss: 9.5416


/home/tarik/miniconda3/envs/catapult/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [4/10], Step [1/4], Loss: 9.2689
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [5/10], Step [1/4], Loss: 10.6421
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [6/10], Step [1/4], Loss: 11.0555
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [7/10], Step [1/4], Loss: 10.3014
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [8/10], Step [1/4], Loss: 13.5884
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32, 36, 99])
Fold [1/5], Epoch [9/10], Step [1/4], Loss: 10.0943
torch.Size([32, 36, 99])
torch.Size([32, 36, 99])
F1-score (micro): 0.0000
torch.Size([32

In [75]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
eval_loss = 0

f1_metric = MulticlassF1Score(num_classes=3, average='micro').to(device)

with torch.no_grad():  # Disable gradient computation for evaluation
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        logits = model(inputs)
        preds = torch.argmax(logits, dim=1)

        # Update the metric
        f1_metric.update(preds, targets)

# Compute final F1
f1_score = f1_metric.compute()
print(f"F1-score (micro): {f1_score.item():.4f}")

# Reset for next epoch
f1_metric.reset()

torch.Size([12, 36, 33, 4])


ValueError: GRU: Expected input to be 2D or 3D, got 4D instead